# 9. 텐서플로 시작하기

In [2]:
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

## 9.2 첫 번째 계산 그래프를 만들어 세션에서 실행하기

In [3]:
import tensorflow as tf

In [4]:
reset_graph()
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x * x * y + y + 2

- 실제로는 어떤 계산도 수행하지 않는다. 단지 계산 그래프만 만들 뿐이다. 사실 변수조차도 초기화되지 않는다. 이 계산 그래프를 평가하려면 텐서플로 **세션**을 시작하고 변수를 초기화한 다음 f를 **평가**(evaluation)해야 한다.

- 텐서플로 세션은 연산을 CPU나 GPU같은 **장치**에 올리고 실행하는 것을 도와주며 모든 변숫값을 가지고 있다. (분산 환경에서는 서버)

세션 만들고 변수를 초기화한 다음 f를 평가하고 세션을 닫기(자원 해제)

In [5]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print (result)
sess.close()

42


- 매번 sess.run() 을 번복하기 번거롭다. with를 사용하자

In [6]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

- 각 변수의 초기화를 일일이 실행하는 대신 global_variables_initializer() 함수를 사용할 수 있다. 초기화를 바로 수행하지 않고 계산 그래프가 실행될 때 모든 변수를 초기화할 노드를 생성한다.

In [7]:
init = tf.global_variables_initializer() # init 노드 준비

with tf.Session() as sess:
    init.run()
    result = f.eval()

- 보통 텐서플로 프로그램은 계산 그래프를 만드는 **구성 단계**, 이 그래프를 실행하는 **실행 단계**로 나뉜다.
- 구성 단계에서는 훈련에 필요한 계산과 머신러닝 모델을 표현한 계산 그래프를 만든다.
- 실행 단계에서는 훈련 스텝을 반복해서 평가하고 모델 파라미터를 점진적으로 개선하기 위해 반복 루프를 수행한다.

## 9.3 계산 그래프 관리

노드를 만들면 자동으로 기본 계산 그래프에 추가된다.

In [8]:
x1 = tf.Variable(1)

In [9]:
x1.graph is tf.get_default_graph()

True

독립적인 계산 그래프를 여러개 만들어야 할 때는 새로운 Graph 객체를 만들어 with 블록 안에서 임시로 이를 기본 계산 그래프로 사용할 수 있다.

In [10]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [11]:
x2.graph is graph

True

In [12]:
x2.graph is tf.get_default_graph()

False

## 9.4 노드 값의 생애주기

한 노드를 평가할 때 텐서플로는 이 노드가 의존하고 있는 다른 노드들을 자동으로 찾아 먼저 평가한다.

In [13]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print (y.eval()) # 10
    print (z.eval()) # 15

10
15


- 먼저 간단한 그래프를 정의한다.
- 이후 세션을 시작하고 y를 평가하기 위해 계산 그래프를 실행한다.
- 텐서플로우는 y가 x에 의존한다는 것과, x는 w에 의존한다는 것을 감지한다.
- w를 평가하고, 그 다음에 x를, 그 다음에 y를 평가해서 y 값을 반환한다.
- z를 평가하기 위해 그래프를 실행한다.
- 텐서플로우는 w와 x를 먼저 평가해야 한다는 것을 감지한다.
- ** 이전에 평가된 w와 x를 재사용 하지 않는다.** (w와 x를 두번 평가한다.)
- 모든 노드의 값을 계산 그래프 실행 간에 유지되지 않는다. 다만 변숫값은 예외이며 그래프 실행간에도 세션에 의해 유지된다.
- (변수는 초기화 될때 일생이 시작되고 세션이 종료될 때까지 남아 있다.)

w와 x를 두번 평가하지 않고 y와 z를 효율적으로 평가하려면 텐서플로가 한 번의 그래프 실행에서 y와 z를 모두 평가하도록 만들어야 한다.

In [14]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print (y_val)
    print (z_val)

10
15


## 9.5 텐서플로를 이용한 선형 회귀

- 상수와 변수 연산은 입력이 없다.(**소스 연산** 이라고 한다.)
- 입력과 출력은 **텐서**라는 다차원 배열이다.

- 데이터셋을 추출하고 모든 훈련 샘플에 편향에 대한 입력($x_0$ = 1)을 추가한다. (이 부분은 넘파이를 사용하므로 즉시 실행됨)
- 텐서플로 상수 노드 x와 y를 만들고 데이터와 타깃을 담는다.
- 텐서플로에서 행렬 연산을 사용해 theta를 정의한다.
- 행렬 함수는 transpose(), matmul(), matrix_inverse()인데 앞서 언급한 것처럼 계산을 즉각 수행하지는 않는다. 다만 그래프가 실행될 때 계싼을 수행할 노드를 생성한다.
- 마지막에 코드는 세션을 만들고 theta를 평가한다.

In [15]:
np.c_[np.array([1,2,3]), np.array([4,5,6])]

array([[1, 4],
       [2, 5],
       [3, 6]])

In [16]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

- GPU가 있는 경우 텐서플로가 자동으로 이 코드를 GPU에서 실행한다.

## 9.6 경사 하강법 구현

정규방정식 대신 경사하강법을 사용해 보자. 먼저 그래디언트를 수동으로 계산해 보고 그 다음에 텐서플로의 자동 미분 기능을 사용해 그래디언트를 자동으로 계산해 보자. 마지막으로 텐서플로에 내장된 옵티마이저(optimizer)를 사용하자.

```
CAUTION 경사 하강법을 사용할 때는 입력 벡터를 정규화하는 것이 중요하다. 그렇지 않으면 훈련 속도가 매우 느려진다. 정규화는 텐서플로나 넘파이, 사이킷런의 StandardScaler 또는 선호하는 다른 도구를 사용해도 가능하다.
```

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

### 9.6.1 직접 그래디언트 계산

- random_uniform() 함수는 난수를 담은 텐서를 생성하는 노드를 그래프에 생성한다. 넘파이의 rand() 함수처럼 크기와 난수의 범위를 입력받는다.
- assign() 함수는 변수에 새로운 값을 할당하는 노드를 생성한다. 여기서는 배치 경사 하강법의 스텝을 구현한다.
- 반복 루프는 훈련 단계를 계속 반복해서 실행하고(n_epoch 만큼), 100번 반복마다 현재의 평균제곱 에러(코드에서 mse 변수)를 출력한다. MSE는 매 반복에서 값이 줄어들어야 한다.
- reduce_mean() 함수는 텐서의 평균을 계산하는 노드를 그래프에 추가한다.
- 만약 assign() 함수를 사용하지 않고 theta = theta - learning_rate * gradients 라고 쓰면 theta는 더 이상 변수가 아니라 tf.subtact() 연산의 출력을 가리키는 텐서(tf.Tensor)가 된다.
- 텐서플로의 변수와 텐서 객체를 계산 그래프와 연산 노드를 가리키는 일종의 핸들이라고 생각하자.

In [18]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print ("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE = 10.511495
Epoch 100 MSE = 0.8598611
Epoch 200 MSE = 0.66734505
Epoch 300 MSE = 0.63428414
Epoch 400 MSE = 0.6109767
Epoch 500 MSE = 0.5929796
Epoch 600 MSE = 0.57897323
Epoch 700 MSE = 0.56801564
Epoch 800 MSE = 0.559399
Epoch 900 MSE = 0.55259025


In [19]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

### 9.6.2 자동 미분 사용

텐서프롤의 자동 미분 기능

In [20]:
gradients = tf.gradients(mse, [theta])[0]

In [21]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print ("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE = 3.6706364
Epoch 100 MSE = 0.6512498
Epoch 200 MSE = 0.5932549
Epoch 300 MSE = 0.5759268
Epoch 400 MSE = 0.5636643
Epoch 500 MSE = 0.5544886
Epoch 600 MSE = 0.54757357
Epoch 700 MSE = 0.5423368
Epoch 800 MSE = 0.53835195
Epoch 900 MSE = 0.5353051


- gradients() 함수는 하나의 연산(여기서는 mse)과 변수 리스트(여기서는 theta 하나)를 받아 각 변수에 대한 연산의 그래디언트를 계산하는 새로운 연산을 만든다. (변수당 하나씩)

- gradients 노드는 theta에 대한 MSE의 그래디언트 벡터를 계산한다.

- **후진 자동 미분**(reverse-mode autodiff)를 통해 모든 입력에 대한 출력의 편미분을 $n_{output}$ + 1 그래프 순회안에 모두 계산한다.

### 9.6.3 옵티마이저 사용

텐서플로는 자동으로 그래디언트를 계산해 주지만 더 쉬운 방법이 있다.

- 텐서플로는 경사 하강법 옵티마이저를 포함하여 여러 가지 내장 옵티마이저를 제공한다.

In [22]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

- 다른 옵티마이저를 사용하고 싶으면 한 줄만 바꾸면 된다.

In [22]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

코드

In [23]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print ("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE = 12.408011
Epoch 100 MSE = 0.75519687
Epoch 200 MSE = 0.5420873
Epoch 300 MSE = 0.5331699
Epoch 400 MSE = 0.5305383
Epoch 500 MSE = 0.5287961
Epoch 600 MSE = 0.52754897
Epoch 700 MSE = 0.52664983
Epoch 800 MSE = 0.52600086
Epoch 900 MSE = 0.5255331


## 9.7 훈련 알고리즘에 데이터 주입

- 미니배치 경사 하강법을 적용하려면 매 반복에서 X와 y를 다음번 미니배치로 바꾸어 주어야 한다. 이를 위해 플레이스홀더 노드를 사용한다.
- 크기를 지정하여 강제할 수 있고, 차원을 None으로 설정하면 어떤 크기도 가능하다는 뜻이 된다.

- 아래 코드에서 A는 랭크가 2(즉, 2차원)이고 열은 3개여야 한다. 하지만 행의 개수는 상관없다.

In [26]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print (B_val_1)

print (B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


미니배치 경사 하강법을 위해서 기존 코드를 수정하자.

In [27]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

배치 크기와 전체 배치 횟수를 정의한다

In [28]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [29]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown, 20640을 최대로 100개 뽑아내기
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

In [30]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown, 20640을 최대로 100개 뽑아내기
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

n_epochs = 1000
learning_rate = 0.01

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='theta')

y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    
    for epoch in range(n_epochs):
        
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})        
        
        if epoch % 100 == 0:
            print ("Epoch", epoch, "MSE =", mse.eval(feed_dict={X:scaled_housing_data_plus_bias,
                                                                y:housing.target.reshape(-1, 1)}))            
    
    best_theta = theta.eval()

Epoch 0 MSE = 0.5658386
Epoch 100 MSE = 0.5247129
Epoch 200 MSE = 0.56764245
Epoch 300 MSE = 0.5250596
Epoch 400 MSE = 0.5438981
Epoch 500 MSE = 0.529258
Epoch 600 MSE = 0.52681017
Epoch 700 MSE = 0.52764094
Epoch 800 MSE = 0.5245456
Epoch 900 MSE = 0.5289343


In [31]:
best_theta

array([[ 2.0714476 ],
       [ 0.8462012 ],
       [ 0.11558535],
       [-0.26835832],
       [ 0.32982782],
       [ 0.00608358],
       [ 0.07052915],
       [-0.87988573],
       [-0.8634251 ]], dtype=float32)

In [29]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown, 20640을 최대로 100개 뽑아내기
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

## 9.8 모델 저장과 복원

- 모델을 훈련시키고 나면 필요할 때 다시 쓸 수 있도록 모델 파라미터를 디스크에 저장해야 한다. 또는 훈련하는 동안 일정한 간격으로 체크포인트를 저장해두면 컴퓨터가 훈련 중간에 문제를 일으켜도 처음부터 다시 시작하지 않고 마지막 체크포인트부터 이어나갈 수 있다.

- 텐서플로에서 모델을 저장하기 위해서는, 구성 단계의 끝에서 (모든 변수 노드를 생성한 후) Saver 노드를 추가하고, 실행 단계에서 모델을 저장하고 싶을 때 save() 메서드에 세션과 체크파인트 파일의 경로를 전달하여 호출하면 된다.

In [32]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

y_pred = tf.matmul(X, theta, name="predictions")                                     
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0: # 100번의 에포크마다 체크포인트 저장하기
            print("Epoch", epoch, "MSE =", mse.eval())  
            save_path = saver.save(sess, "./tmp/my_model.ckpt")
            
        sess.run(training_op)        
        
    best_theta = theta.eval()
    save_path = saver.save(sess, "./tmp/my_model_final.ckpt")

Epoch 0 MSE = 9.161543
Epoch 100 MSE = 0.7145006
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.5555719
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436362
Epoch 600 MSE = 0.5396294
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.5340678
Epoch 900 MSE = 0.5321474


In [33]:
save_path

'./tmp/my_model_final.ckpt'

In [34]:
best_theta

array([[ 2.0685525 ],
       [ 0.8874027 ],
       [ 0.14401658],
       [-0.34770882],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269556],
       [-0.6614528 ],
       [-0.6375277 ]], dtype=float32)

모델 복원하기

- 구성 단계의 끝에서 Saver 노드를 생성하고, 실행 단계를 시작할 때 init 노드를 사용하여 변수를 초기화하는 대신, saver 객체의 restore() 메서드를 호출한다.

In [35]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

y_pred = tf.matmul(X, theta, name="predictions")                                     
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, "./tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from ./tmp/my_model_final.ckpt


- 값이 같은지 확인해 보자

In [36]:
np.allclose(best_theta, best_theta_restored)

True

- 복원할 변수를 지정하거나 별도의 이름을 사용할 수 있다. 가령 여기서는 theta 변수만 weights란 이름으로 저장하고 복원할 것이다.

In [37]:
saver = tf.train.Saver({"weights": theta})

- save() 메서드는 기본적으로 .meta 확장자를 가진 동일 이름의 두 번째 파일에 그래프 구조를 저장한다.
- ```tf.train.import_meta_graph```를 이용해 이 그래프 구조를 읽어 들일 수 있다. 이 함수는 그래프를 기본 그래프로 추가되며, 반환된 Saver 인스턴스로 그래프의 상태를 복원하는데 이용할수 있다.

In [38]:
tf.reset_default_graph()
# 빈 그래프로 시작한다.

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # not shown in the book
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [39]:
np.allclose(best_theta, best_theta_restored)

True

## 9.9 텐서보드로 그래프와 학습 곡선 시각화하기

- 프로그램을 실행할 때마다 다른 로그 디렉토리를 사용하도록 해야 한다.
- 그렇지 않으면 프로그램을 실행할 때마다 만들어진 통계가 합쳐져서 텐서보드 그래프가 엉망이 된다.
- 로그 디렉터리 이름에 타임스템프를 포함하면 간단하게 해결된다.

In [61]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = 'tf_logs'

logdir = f"{root_logdir}/run-{now}/"

In [77]:
logdir

'tf_logs/run-20180719120900/'

In [62]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42), name='theta')

y_pred = tf.matmul(X, theta, name='predictions')
errors = y_pred - y
mse = tf.reduce_mean(tf.square(errors), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [63]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

- 첫 번째 줄은 MSE 값을 평가하고 그것을 **서머리**라 부르는 텐서보드가 인식하는 이진 로그 문자열에 쓰기 위한 노드를 그래프에 추가한다.
- 둘째 줄은 FileWriter 객체를 만들어 로그 디렉터리에 있는 로그 파일에 서머리를 기록한다.

In [64]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [65]:
n_batches

207

In [58]:
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown
    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                                     # not shown

마지막으로 프로그램 끝에서 FileWriter 객체를 닫는다.

In [75]:
file_writer.close()

In [76]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

## 9.10 이름 범위

신경망처럼 복잡한 모델을 다룰 때는 계산 그래프가 수천 개의 노드로 인해 어질러지기 쉽다. 이를 피하려면 **이름 범위**를 만들어 관련 있는 노드들을 그룹으로 묶어야 한다. 가령 이전 코드를 수정해 "loss" 이름 범위 안에 있는 error와 mse를 정의해 보자.

In [78]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

이제 이 범위 안에 있는 모든 연산의 이름에는 'loss/' 접두사가 붙는다.

In [79]:
error.op.name

'loss/sub'

In [80]:
mse.op.name

'loss/mse'

In [89]:
mse.op.name

'loss/mse'

In [90]:
reset_graph()

a1 = tf.Variable(0, name='a')    # name == 'a'
a2 = tf.Variable(0, name='a')    # name == 'a_1'

with tf.name_scope('param'):     # name == 'param'
    a3 = tf.Variable(0, name='a')# name == 'param/a'
    
with tf.name_scope('param'):     # name == 'param_1'
    a4 = tf.Variable(0, name='a')# name == 'param_1/a'

for node in (a1, a2, a3, a4):
    print (node.op.name)

a
a_1
param/a
param_1/a


## 9.11 모듈화

In [91]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

Much better, using a function to build the ReLUs:

In [112]:
reset_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    return tf.maximum(z, 0., name='relu')

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

Even better using name scopes:

In [114]:
def relu(X):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        return tf.maximum(z, 0., name='relu')

In [115]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

## 9.12 변수 공유

```threshold``` 변수를 공유하려면, ```relu()``` 함수 바깥에 정의하고 이 변수를 매개변수로서 전달하는 것이 가장 기본적인 방법이다.

In [116]:
reset_graph()

def relu(X, threshold):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

relu()를 맨 처음 호출할 때 함수의 속성으로 다음과 같이 공유 변수를 지정할 수 있다.

In [124]:
def relu(X):
    with tf.name_scope('relu'):
        if not hasattr(relu, 'threshold'):
            relu.threshold = tf.Variable(0.0, name='threshold')
        
        w_shape = int(X.get_shape()[1]), 1
        w = tf.Variable(tf.random_normal(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        
        return tf.maximum(z, relu.threshold, name='max')

In [125]:
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name='output')

만약 이 변수가 이전의 get\_variable() 호출에서 이미 생성되었다면 이 코드는 예외를 발생시킬 것이다. 이런 동작은 실수로 변수가 사용되는 것을 막아준다. 변수를 재사용하고 싶다면 명시적으로 변수 범위의 reuse 속성을 True로 지정해야 한다.

In [127]:
reset_graph()

with tf.variable_scope('relu'):
    threshold = tf.get_variable('threshold', shape=(), # shape ()이므로 스칼라 변수이고 초깃값은 0.0이다.
                               initializer=tf.constant_initializer(0.0))

reuse 속성 True

In [129]:
with tf.variable_scope('relu', reuse=True):
    threshold = tf.get_variable('threshold')

이 코드는 이미 존재하는 relu/threshold 변수를 가져오며, 존재하지 않거나 get\_variable()로 만들지 않은 변수일 경우에는 예외가 발생한다. 또한 변수 범위의 블록 안에서 reuse\_variable() 메서드를 호출하여 reuse 속성을 True로 설정할 수도 있다.

In [130]:
with tf.variable_scope('relu') as scope:
    scope.reuse_variables()
    threshold = tf.get_variable('threshold')

In [131]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

In [132]:
file_writer = tf.summary.FileWriter("logs/relu6", tf.get_default_graph())
file_writer.close()

**NOTE** get\_variable()로 만든 변수는 항상 variable_scope의 이름을 접두사로 사용한다. 예를 들어 relu/threshold 와 같다. 다른 모든 노드에 대해서는 변수 범위가 새로운 이름 범위처럼 작동한다.

In [133]:
reset_graph()

def relu(X):
    with tf.variable_scope('relu'):
        threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
with tf.variable_scope("", default_name="") as scope:
    first_relu = relu(X)
    scope.reuse_variables()
    relus = [first_relu] + [relu(X) for i in range(4)]
output = tf.add_n(relus, name='output')

file_writer = tf.summary.FileWriter('logs/relu8', tf.get_default_graph())
file_writer.close()

threshold 변수를 함수 안에서 생성하고 그 다음부터 호출될 때는 이 변수를 재사용하도록 코드를 짜 보자

In [137]:
reset_graph()

def relu(X):
    threshold = tf.get_variable('threshold', shape=(), initializer=tf.constant_initializer(0.0))
    w_shape = int(X.get_shape()[1]), 1
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
    b = tf.Variable(0.0, name="bias")                           # not shown
    z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = []
for relu_index in range(5):
    with tf.variable_scope('relu', reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name='output')

file_writer = tf.summary.FileWriter("logs/relu9", tf.get_default_graph())
file_writer.close()

## 기타

In [145]:
reset_graph()

with tf.variable_scope("my_scope"):
    x0 = tf.get_variable("x", shape=(), initializer=tf.constant_initializer(0.))
    x1 = tf.Variable(0., name="x")
    x2 = tf.Variable(0., name="x")

with tf.variable_scope("my_scope", reuse=True):
    x3 = tf.get_variable("x")
    x4 = tf.Variable(0., name="x")

with tf.variable_scope("", default_name="", reuse=True):
    x5 = tf.get_variable("my_scope/x")

print("x0:", x0.op.name)
print("x1:", x1.op.name)
print("x2:", x2.op.name)
print("x3:", x3.op.name)
print("x4:", x4.op.name)
print("x5:", x5.op.name)
print(x0 is x3 and x3 is x5)

x0: my_scope/x
x1: my_scope/x_1
x2: my_scope/x_2
x3: my_scope/x
x4: my_scope_1/x
x5: my_scope/x
True


첫 번째 ```variable_scope()``` 블락이 이름을 ```my_scope/x``` 으로한 공유 변수 ```x0``` 생성한다. 공유된 변수들을 제외한 모든 연산에서(non-shared variables를 포함하여), 변수 scope는 마치 name scope처럼 동작한다. 그렇기 때문에 ```x1```과 ```x2```가 ```my_scope/x_1``` 그리고 ```my_scope/x_2```라고 되어있는 것이다.

두 번째 ```variable_scope()``` 블락이 ```my_scope```에 있는 공유 변수들을 다시 사용한다. 그렇기 때문에 ```x0```이 ```x3```이 된다. 다시 한번 말하지만, 공유된 변수를 제외한 모든 연산이 이름 scope와 같이 행동하기 때문에, scope의 이름이 텐서 플로우에 의해서 ```my_scope_1```이 된다. 그리고 변수 ```x4```는 ```my_scope_1/x```이 된다.

세번째 블락은 공유된 변수 변수 ```my_scope/x```를 다루는 새로운 방법을 알려준다.